# Learn Keras for Deep Neural Networks
## Chapter 6 - The Path Foward

Consolidating few reading links and additional references for the readers of the book to explore additional references and other advanced topics 

### Recommeded books for Machine Learning with Python

* Mastering Machine Learning with Python in 6 steps - https://www.apress.com/us/book/9781484228654
* Pactical Machine Learning with Python - https://www.apress.com/us/book/9781484232064


### Recommeded books for Deep Learning with Python/Tensorflow
* Applied Deep Learning -  https://www.apress.com/us/book/9781484237892
* Pro Deep Learning with Tensorflow - https://www.apress.com/us/book/9781484230954



### Reading Links Convolutional Neural Networks (CNN)
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/
* https://medium.freecodecamp.org/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050


### Reading Links for Recurrent Neural Networks
* https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21


### Reading Links for Image Captioning  (CNN + RNN)
* https://towardsdatascience.com/image-captioning-in-deep-learning-9cd23fb4d8d2
* https://machinelearningmastery.com/introduction-neural-machine-translation/
* https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd


### Reading Links for Generative Adversarial Networks 
* https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39
* https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f
* https://towardsdatascience.com/understanding-generative-adversarial-networks-4dafc963f2ef